In [1]:
# some primatives

import numpy as np
import math

def mylog2(n):
    if n == 0:
        return 0
    else:
        return math.log2(n)
def raise_(ex):
    raise ex


def loop_till_fix(equals):
    def loopaa(F):
        def apply_till_fix(x):
            while True:
                x1 = F(x)
                if equals (x1)(x):
                    return x
                x = x1
        return apply_till_fix
    return loopaa


# problem specific definitions


def merge(left, right):
    if np.isnan(left).all() or np.isnan(right).all():
        return np.nan
    merged = []
    i = j = 0
    # standard two-pointer merge logic
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            merged.append(left[i])
            i += 1
        else:
            merged.append(right[j])
            j += 1

    # append leftovers
    merged.extend(left[i:])
    merged.extend(right[j:])
    #print(type(merged)) # debug line
    return tuple(merged)


def hl(arr):
    mid = (len(arr) + 1) // 2   # ceil division
    return arr[:mid]


def hr(arr):
    mid = (len(arr) + 1) // 2
    return arr[mid:]


In [2]:
#h = lambda a : lambda n : set((a,)) if n==0 else (h(a)(n-1) | set(map(hl, tuple(h(a)(n-1)))) | set(map(hr, tuple(h(a)(n-1)))))

def h(a):
    def foo(n):
        if n==0 :
            return set((a,))
        else:
            set1 = h(a)(n-1)
            set2 = set(map(hl, h(a)(n-1)))
            set3 = set(map(hr, h(a)(n-1)))
            return set1 | set2 | set3
    return foo


In [3]:
# defining equality over functions on ha
# eq a x1 x2 := {x1=x2}
# h(a)(math.ceil(math.log2(len(a))))
def equ (domain):
    def foo (x1):
        def goo (x2):            
            for arr in domain:
    
                if x1(arr) != x2(arr):
                    return False
            return True
        return goo
    return foo

def loop(input_array):
    domain = h(input_array)(math.ceil(mylog2(len(input_array))))
    return loop_till_fix(equ(domain))

In [4]:
test_arr = (9,6,5,3,1,8,7,2,4)

h(test_arr)(4)

{(),
 (1,),
 (2,),
 (2, 4),
 (3,),
 (3, 1),
 (4,),
 (5,),
 (6,),
 (7,),
 (8,),
 (8, 7),
 (8, 7, 2, 4),
 (9,),
 (9, 6),
 (9, 6, 5),
 (9, 6, 5, 3, 1),
 (9, 6, 5, 3, 1, 8, 7, 2, 4)}

In [5]:
h(())(2)

{()}

In [6]:
h(test_arr)(0)

{(9, 6, 5, 3, 1, 8, 7, 2, 4)}

# Mathematical Model
$
m(a) :=
\begin{cases}
a & \text{if } |a| \leq 1 \\[6pt]
\text{merge}(m(h_l a), m(h_r a)) & \text{else}
\end{cases}
$  
# As mapcode
## Primatives
1. **array data structure**. Also size of array $a$ is given by $|a|$  
2. **merge function:** $\text{merge}(a,b)$ returns sorted array containing elements of $a$ and $b$  
3. **list halving:** $h_l a$ returns an array with first $⌈\frac{|a|}{2}⌉$ elements of $a$ while $h_r a$ returns the rest  
4. **given an array $a$ define a family of sets $h^n_a$** such that  
$$
h^n_a := \begin{cases}
a & \text{if } n = 0 \\[6pt]
h^{n-1}_a \cup \{h_lx \mid x \in h^{n-1}_a\} \cup \{h_rx \mid x \in h^{n-1}_a\} & \text{else}
\end{cases}
$$

## Mapcode definitions  
$I := \{\text{set of arrays}\}$  
$X := h^{⌈\log_2 |a|⌉}_a \rightarrow \{\text{set of arrays}\} $  
$A := \{\text{set of arrays}\}$  

Define $\rho : I \rightarrow X$ ,  
$F : X \rightarrow X$,  
$\pi : X \rightarrow A$  
such that  
$\rho(a) := \{ \text{arr} \rightarrow ⊥ \text{ | } \text{arr} \in h^{⌈\log_2 |a|⌉}_a\}$  
$
F(x)(\text{arr}) :=
\begin{cases}
\text{arr} & \text{if } |\text{arr}| \leq 1 \\[6pt]
\text{merge}(x (h_l \text{arr}),x (h_r \text{arr})) & \text{if } x (h_l \text{arr}) \text{ , } x (h_r \text{arr}) \neq \bot\\[6pt]
\bot & \text{else}
\end{cases}
$  
$(\pi(a))(x) := x(a)$

# Implementation

In [7]:
rho = lambda a : lambda arr : np.nan if arr in h(a)(math.ceil(mylog2(len(a)))) else raise_(ValueError(f"out of bounds of array {a}'s tree"))
F = lambda x : lambda arr : arr if len(arr) <= 1 else merge(x(hl(arr)) , x(hr(arr)))
pi = lambda a : lambda x : x(a)

mapcode = lambda a : pi(a)(loop(a)(F)(rho(a)))

In [8]:
sample_arr = (9,6,5,3,1,8,7,2,4)

# Example

In [9]:
mapcode(sample_arr)

(1, 2, 3, 4, 5, 6, 7, 8, 9)

In [10]:
print(mapcode(())) # empty array
print(mapcode((2,)))
print(mapcode((0,)), type(mapcode((0,))))
print(mapcode((0,2)), type(mapcode((0,2))))
print(mapcode((2,0)))
print(mapcode(mapcode(sample_arr))) # already sorted array


()
(2,)
(0,) <class 'tuple'>
(0, 2) <class 'tuple'>
(0, 2)
(1, 2, 3, 4, 5, 6, 7, 8, 9)


# Visualisation with an array of size 9

In [11]:
#  sample_arr = (9,6,5,3,1,8,7,2,4)
arr_of_x = [rho(sample_arr)] # why error if i pass in (9,6,5,3,1,8,7,2,4) to rho insteaad of sample_arr
domain_of_x = h(sample_arr)(math.ceil(mylog2(len(sample_arr))))
for nn in range(0,10):
    yolo = F(arr_of_x[nn])
    arr_of_x.append(yolo)
    if equ(domain_of_x)(yolo)(arr_of_x[nn]):
        print (f"fixed point reached at x_{nn}")
        break

sorted_list_of_domain_x = sorted(list(domain_of_x), key=len, reverse=True)
count = 0
for xx in arr_of_x:
    print(f"\n\n{count}.--- {xx} ----")
    for node in sorted_list_of_domain_x:
        print(f"{node} : {xx(node)}")
    count = count + 1

fixed point reached at x_5


0.--- <function <lambda>.<locals>.<lambda> at 0x7ff5a02754e0> ----
(9, 6, 5, 3, 1, 8, 7, 2, 4) : nan
(9, 6, 5, 3, 1) : nan
(8, 7, 2, 4) : nan
(9, 6, 5) : nan
(2, 4) : nan
(3, 1) : nan
(8, 7) : nan
(9, 6) : nan
(2,) : nan
(5,) : nan
(8,) : nan
(3,) : nan
(9,) : nan
(6,) : nan
(4,) : nan
(1,) : nan
(7,) : nan
() : nan


1.--- <function <lambda>.<locals>.<lambda> at 0x7ff5a0275440> ----
(9, 6, 5, 3, 1, 8, 7, 2, 4) : nan
(9, 6, 5, 3, 1) : nan
(8, 7, 2, 4) : nan
(9, 6, 5) : nan
(2, 4) : nan
(3, 1) : nan
(8, 7) : nan
(9, 6) : nan
(2,) : (2,)
(5,) : (5,)
(8,) : (8,)
(3,) : (3,)
(9,) : (9,)
(6,) : (6,)
(4,) : (4,)
(1,) : (1,)
(7,) : (7,)
() : ()


2.--- <function <lambda>.<locals>.<lambda> at 0x7ff5a0274f40> ----
(9, 6, 5, 3, 1, 8, 7, 2, 4) : nan
(9, 6, 5, 3, 1) : nan
(8, 7, 2, 4) : nan
(9, 6, 5) : nan
(2, 4) : (2, 4)
(3, 1) : (1, 3)
(8, 7) : (7, 8)
(9, 6) : (6, 9)
(2,) : (2,)
(5,) : (5,)
(8,) : (8,)
(3,) : (3,)
(9,) : (9,)
(6,) : (6,)
(4,) : (4,)
(1,) : (1,)
(7,) 

# Beeg Example

In [12]:
import random

arro = tuple(random.randint(0, 21) for _ in range(100))

In [13]:
arr_of_x = [rho(arro)]

for nn in range(0,10):
    arr_of_x.append(F(arr_of_x[nn]))

count = 0
for xx in arr_of_x:
    print(f"\n\n{count}.--- {xx} ----")
    for node in h(arro)(math.ceil(mylog2(len(arro)))):
        print(xx(node))
    count = count + 1



0.--- <function <lambda>.<locals>.<lambda> at 0x7ff5a0274ea0> ----
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


1.--- <function <lambda>.<locals>.<lambda> at 0x7ff5a0275080> ----
nan
nan
nan
(20,)
nan
nan
nan
nan
nan
nan
(13,)
nan
nan
nan
nan
nan
nan
nan
nan
(6,)
nan
nan
nan
nan
nan
nan
nan
nan
(15,)
nan
nan
nan
nan
()
nan
nan
(8,)
nan
(17,)
nan
nan
nan
nan
nan
nan
(1,)
nan
(10,)
nan
(19,)
nan
nan
nan
nan
(3,)
nan
nan
(12,)
(21,)
nan
nan
nan
nan
nan
(5,)
nan
(14,)
nan
nan
nan
nan
nan
nan
nan
nan
nan
(7,)
(16,)
nan
nan
nan
(0,)
(9,)
nan
(18,)
nan
nan
n

## Bound check test

In [14]:
xqa = rho((9,6,5,3,1,8,7,2,4))
xqa((3,5))

ValueError: out of bounds of array (9, 6, 5, 3, 1, 8, 7, 2, 4)'s tree